In [3]:
#引用套件
!pip install boto3
import boto3
from boto3.dynamodb.conditions import Key, Attr

In [4]:
#連線 dynamodb 
sqs = boto3.resource(
    'sqs',
    endpoint_url='http://cc104.sqs.local:9324',    # sqs URL
    region_name='my_region',                       # aws 區域
    aws_access_key_id='my_access_key',             
    aws_secret_access_key='my_secret_key',         
    verify=False)

In [5]:
# Create the queue. This returns an SQS.Queue instance
#queue = sqs.create_queue(QueueName='test', Attributes={'DelaySeconds': '1'})
queue = sqs.create_queue(QueueName='test3')

# You can now access identifiers and attributes
print(queue.url)
#print(queue.attributes.get('DelaySeconds'))

http://cc104.sqs.local:9324/queue/test3


In [7]:
# Get the queue. This returns an SQS.Queue instance
queue = sqs.get_queue_by_name(QueueName='test')

# You can now access identifiers and attributes
# print(queue.url)
# print(queue.attributes.get('DelaySeconds'))
# #It is also possible to list all of your existing queues:

# Print out each queue name, which is part of its ARN
for queue in sqs.queues.all():
    print(queue.url)

http://cc104.sqs.local:9324/queue/test


In [15]:
# Get the queue
queue = sqs.get_queue_by_name(QueueName='test')

# Create a new message
response = queue.send_message(MessageBody='this is a queue test')

# The response is NOT a resource, but gives you a message ID and MD5
print(response.get('MessageId'))
print(response.get('MD5OfMessageBody'))



e5198fb2-f8d6-4002-a870-e2664be1e5ff
8483ff2e5e36d660daf72c88ba7c3bbe


In [ ]:
# You can also create messages with custom attributes
queue.send_message(MessageBody='boto3', MessageAttributes={
    'Author': {
        'StringValue': 'Daniel',
        'DataType': 'String'
    }
})

In [6]:
# Messages can also be sent in batches. For example, 
# sending the two messages described above in a single request would look like the following
# Get the queue
queue = sqs.get_queue_by_name(QueueName='test3')

response = queue.send_messages(Entries=[
    {
        'Id': '1',
        'MessageBody': 'this is a queue test1'
    },
    {
        'Id': '2',
        'MessageBody': 'this is a queue test2',
        'MessageAttributes': {
            'Author': {
                'StringValue': 'hahaha',
                'DataType': 'String'
            }
        }
    }
])

# Print out any failures
# print(response.get('Failed'))

In [9]:
# Process messages by printing out body and optional author name
for message in queue.receive_messages(MessageAttributeNames=['Author']):
    # Get the custom author message attribute if it was set
    author_text = ''
    if message.message_attributes is not None:
        author_name = message.message_attributes.get('Author').get('StringValue')
        if author_name:
            author_text = ' ({0})'.format(author_name)
    print('Hello, {0}!{1}'.format(message.body, author_text))
    
    
# Let the queue know that the message is processed
    message.delete()